In [31]:
import re
import pandas as pd
import numpy as np
import sqlite3
import os 

# change directory 
os.chdir(r'C:\Users\Rodzinka\Desktop\siatka_23')

# read database, change path and database name if needed
conn = sqlite3.connect('data/database_players_allteams.db')
cursor = conn.cursor()
result = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

In [32]:
# find table names
table_names = sorted(list(zip(*result))[0])

# define teams' names
names = ['Aluron', 'AZS Częstochowa', 'Asseco', 'BBTS', 'Bydgoszcz', 'Kielce', 'Szczecin', 'Czarni', 'Lubin', 
         'Indykpol', 'GKS', 'Węgiel', 'Będzin', 'Wieluń',
         'Lublin', 'Skra', 'Warsza', 'Nysa', 'Suwałki', 'ZAKSA', 'Gdańsk', 'Hermapol'
         'test']

# define years
years = ['2023/2024', '2022/2023', '2021/2022', '2020/2021', '2019/2020', '2018/2019', '2017/2018', '2016/2017', '2015/2016', '2014/2015', '2013/2014', 
        '2012/2013', '2011/2012', '2010/2011']

In [33]:
# create final table
tab = pd.DataFrame(columns=['Height_avg', 'Weight_avg', 'Range_avg', 'Atakujący_Height',
                   'Libero_Height', 'Przyjmujący_Height', 'Rozgrywający_Height',
                   'Środkowy_Height', 'Atakujący_Weight', 'Libero_Weight',
                   'Przyjmujący_Weight', 'Rozgrywający_Weight', 'Środkowy_Weight',
                   'Atakujący_Range', 'Libero_Range', 'Przyjmujący_Range',
                   'Rozgrywający_Range', 'Środkowy_Range', 'Team_name', 'Year'])

for year in  years:
    for name in names: 
        
        # list for storing table names
        names_players = []
        for table_name in table_names:
            
            if table_name.find(year) != -1:
                if table_name.lower().find(name.lower())!= -1:
                    names_players.append(table_name)

        if not names_players:
            continue           
        else:
            #print(names_players)

            # get results for each round
            for name_tab in names_players:
                # read table
                data = pd.read_sql("SELECT * FROM '{}'".format(name_tab), conn)
                
                # average for all players
                data = data.apply(pd.to_numeric, errors='ignore')
                data_temp = data[['Height', 'Weight', 'Range']].mean(axis=0)
                temp = [round(w, 2) for w in data_temp]
                average = pd.DataFrame(columns = ['Height_avg', 'Weight_avg', 'Range_avg'])
                
                average = pd.concat([average, pd.DataFrame([temp], columns=['Height_avg', 'Weight_avg', 'Range_avg'])], ignore_index=False)
                average = average.reset_index(drop=True)
                
                # average for each position
                try:
                    by_position = data.groupby("Position").mean().round(2)
                    by_position=by_position.reset_index()

                    # change data format to wide
                    df_long = pd.melt(by_position, id_vars=['Position'])

                    # transpose data
                    transpose = df_long.T

                    # create unique column names
                    transpose.loc['name'] = transpose.loc['Position'] + '_' + transpose.loc['variable']
                    transpose.drop('Position', inplace=True)
                    transpose.drop('variable', inplace=True)
                    transpose.columns = transpose.iloc[1]
                    transpose.drop('name', inplace=True)
                    transpose = transpose.reset_index(drop=True)
                    
                    # merge created tables
                    data_f = pd.concat([average, transpose], axis=1)
                    data_f['Team_name'] = name
                    data_f['Year'] = year
                    
                    # append results to final table
                    tab = pd.concat([tab, data_f], ignore_index=True)
                    
                except:
                    print(name_tab, 'Could not retrieve data for players')
                    pass
        
tab

Stocznia Szczecin 2018/2019 Could not retrieve data for players


,Height_avg,Weight_avg,Range_avg,Atakujący_Height,Libero_Height,Przyjmujący_Height,Rozgrywający_Height,Środkowy_Height,Atakujący_Weight,Libero_Weight,...,Atakujący_Range,Libero_Range,Przyjmujący_Range,Rozgrywający_Range,Środkowy_Range,Team_name,Year,_Height,_Weight,_Range
0,195.53,91.13,341.20,196.5,182.5,194.75,191.5,202.6,97.5,76.0,...,330.0,330.0,351.75,320.0,350.2,Aluron,2023/2024,NaN,NaN,NaN
1,197.28,86.56,344.78,196.5,183.5,199.4,194.67,201.67,87.0,76.0,...,349.0,319.0,349.2,343.0,349.17,Asseco,2023/2024,NaN,NaN,NaN
2,197.47,89.67,339.27,201.5,186.5,196.75,192.67,205.25,100.5,73.5,...,348.5,318.5,346.25,326.0,348.0,Czarni,2023/2024,NaN,NaN,NaN
3,196.14,89.21,345.14,203.5,181.0,196.0,191.0,202.75,94.0,72.0,...,360.5,319.0,351.5,333.5,350.0,Lubin,2023/2024,NaN,NaN,NaN
4,195.27,87.60,341.33,202.0,179.0,196.5,190.0,202.5,98.0,74.5,...,367.0,316.5,338.83,320.0,361.75,Indykpol,2023/2024,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,184.62,86.12,315.69,200.0,184.5,163.5,191.5,205.25,86.0,80.5,...,349.0,320.0,287.33,342.5,326.0,Węgiel,2010/2011,NaN,NaN,NaN
182,149.00,61.54,235.08,97.5,189.0,194.0,192.5,67.67,0.0,82.0,...,0.0,335.0,343.75,330.5,116.67,Wieluń,2010/2011,NaN,NaN,NaN
183,187.83,87.39,326.67,199.0,184.0,198.4,193.33,171.33,91.33,73.0,...,359.0,328.0,348.6,331.33,289.67,Skra,2010/2011,NaN,NaN,NaN
184,197.92,95.08,343.92,200.0,180.0,195.5,195.0,205.25,96.0,79.0,...,347.5,330.0,347.0,333.5,347.75,Warsza,2010/2011,NaN,NaN,NaN


In [34]:
# remove unnecessary columns 
tab_final = tab.iloc[:,:-3]

In [37]:
# create database for new data
conn3 = sqlite3.connect('data/database_players_averages_all_teams.db')
tab_final.to_sql("Players_prepared", conn3, if_exists='replace', index=False)   

186

In [38]:
# close connection
cursor.close() 
conn.close()
conn3.close()

ProgrammingError: Cannot operate on a closed database.